In [157]:
import numpy as np
import os
import re
from sklearn.naive_bayes import BernoulliNB
import string



'''
def lg(num):
    if num==0:
        return 0
    else:
        return np.log(num)

def whichClass(spam, legit):
    if legit==0:
        return 1 #spam
    elif spam/legit>=1:
        return 1 #spam
    else:
        return 0 #legit
        
# Get all the words from all emails, and separate them by spam and non-spam.

p = "lingspam_public/lemm_stop"
dict_legit = {}
dict_spam = {}
term_freq = {}
spam = 0
legit = 0
pattern = re.compile(r'([a-z]+)', re.I)
for i in range(1,11):
    path = p + "/part" + str(i)
    files = os.listdir(path)
    for file in files:
        dict_count = {}
        txt = open(path +"/"+ file).read()
        words = pattern.findall(txt)
        for word in words:
            dict_count[word] = dict_count.get(word, 0)+1
            term_freq[word] = term_freq.get(word, 0)+1
        for word in dict_count.keys():
            if dict_count.get(word, 0) != 0 and file.find("spm")==-1:
                dict_legit[word] = dict_legit.get(word, 0) + 1
            else:
                dict_spam[word] = dict_spam.get(word, 0) + 1
word_legit = sorted(dict_legit, key = lambda x: dict_legit[x], reverse = True)
word_spam = sorted(dict_spam, key = lambda x: dict_spam[x], reverse = True)
word_ = {}.fromkeys(word_legit+word_spam).keys()

spam = 481
legit = 2412
n = 2893
H = -(legit/n)*np.log(legit/n)-(spam/n)*np.log(spam/n)
class_ = {}

ig = {}
for word in word_list:
    if word=="Subject":
        continue
    N = dict_legit.get(word,0)+dict_spam.get(word,0)
    p_spam = dict_spam.get(word,0)/spam
    p_legit = dict_legit.get(word,0)/legit
    lg_spam = (spam/n)*(p_spam)/(N/n)
    
    H_ = -(spam/n)*(p_spam)*lg(lg_spam)-(spam/n)*(1-p_spam)*lg((spam/n)*(1-p_spam)/(1-N/n))\
        -(legit/n)*(p_legit)*lg((legit/n)*(p_legit)/(N/n))-(legit/n)*(1-p_legit)*lg((legit/n)*(1-p_legit)/(1-N/n))
    ig[word] = H-H_
    
    class_[word] = whichClass((dict_spam.get(word,0)/spam)*(spam/n),(dict_legit.get(word,0)/legit)*(legit/n))
data = sorted(ig, key = lambda x: ig[x], reverse = True)
N_10 = data[0:10]
print(N_10)
N_10 = N_10.reshape(10,1)
N_100 = data[0:100]
N_100 = N_100.reshape(100,1)
N_1000 = data[0:1000]
N_1000 = N_1000.reshape(1000,1)
class_N_10 = []
for word in N_10:
    class_N_10.append(class_[word])
class_N_100 = []
for word in N_100:
    class_N_100.append(class_[word])
class_N_1000 = []
for word in N_1000:
    class_N_1000.append(class_[word])
    
# Bernoulli NB classifier with binary features




'''


def load_emails(path):
    texts = []
    for entry in os.listdir(path):
        is_spam = 0
        if entry[0] == 's':
            is_spam = 1
        with open(os.path.join(path, entry), 'r') as mail:
            next(mail)
            word_count = {}
            words = mail.read().replace('\n', '').split()
            for word in words:
                striped = word.strip(string.punctuation)
                if striped.isalpha():
                    if striped not in word_count:
                        word_count[striped] = 1
                    else:
                        word_count[striped] += 1
            texts.append((word_count, is_spam))
    return texts


def select_feature(data, size):
    word_classes = {}
    spams = 0
    hams = 0
    for word_count, is_spam in data:
        if is_spam:
            spams += 1
        else:
            hams += 1
        for word in word_count:
            if word not in word_classes:
                word_classes[word] = [0, 0]
            word_classes[word][is_spam] += 1
    sorted_features = sorted(word_classes,
                             key=lambda x: conditional(word_classes[x][1], word_classes[x][0], spams, hams),
                             reverse=True)
    return sorted_features[0: min(size, len(sorted_features))]


def conditional(count_spam, count_ham, spams, hams):
    entropy = 0
    if count_spam > 0:
        entropy += count_spam * np.log(count_spam / (count_spam + count_ham))
    if count_ham > 0:
        entropy += count_ham * np.log(count_ham / (count_spam + count_ham))
    if spams - count_spam > 0:
        entropy += (spams - count_spam) * np.log((spams - count_spam) / (spams + hams - count_spam - count_ham))
    if hams - count_ham > 0:
        entropy += (hams - count_ham) * np.log((hams - count_ham) / (spams + hams - count_spam - count_ham))
    return entropy


def evaluate(eval_model, eval_data):
    true_pos = 0
    false_pos = 0
    true_neg = 0
    false_neg = 0
    for text, is_spam in eval_data:
        prediction = eval_model.predict(text)
        if is_spam == 1:
            if prediction == 1:
                true_pos += 1
            else:
                false_neg += 1
        else:
            if prediction == 1:
                false_pos += 1
            else:
                true_neg += 1
    eval_precision = true_pos / (true_pos + false_pos)
    eval_recall = true_pos / (true_pos + false_neg)
    false_pos_rate = false_pos / (false_pos + true_neg)
    false_neg_rate = false_neg / (true_pos + false_neg)
    return eval_precision, eval_recall, false_pos_rate, false_neg_rate






In [158]:
#bernoulli

import numpy as np


# Naive Bayes classifier
class NB:

    def __init__(self, classifier_type):
        if classifier_type not in ['Binomial BF', 'Multinomial BF', 'Multinomial TF']:
            raise ValueError('Invalid classifier type.')
        self.classifier_type = classifier_type
        self.spam_prob = {}
        self.ham_prob = {}

    def fit(self, texts, features):
        spam_dict = {}
        ham_dict = {}
        texts_spam = 0
        texts_ham = 0
        # Laplace smoothing
        for word in features:
            spam_dict[word] = 1
            ham_dict[word] = 1
        for word_count, is_spam in texts:
            if is_spam == 1:
                texts_spam += 1
                for word in word_count:
                    if word in spam_dict:
                        if self.classifier_type[0] == 'B':
                            spam_dict[word] += 1
                        elif self.classifier_type[0] == 'M':
                            spam_dict[word] += word_count[word]
            else:
                texts_ham += 1
                for word in word_count:
                    if word in ham_dict:
                        if self.classifier_type[0] == 'B':
                            ham_dict[word] += 1
                        elif self.classifier_type[0] == 'M':
                            ham_dict[word] += word_count[word]
        if self.classifier_type[0] == 'B':
            spam_count = texts_spam + 2
            ham_count = texts_ham + 2
        else:
            spam_count = sum(spam_dict.values())
            ham_count = sum(ham_dict.values())
        for word in spam_dict:
            self.spam_prob[word] = spam_dict[word] / spam_count
        for word in ham_dict:
            self.ham_prob[word] = ham_dict[word] / ham_count

    def predict(self, text):
        # spam_rate is the prior probability that an email is spam, 0.5 by default
        predict_spam = 0
        predict_ham = 0
        # Binomial BF
        if self.classifier_type[0] == 'B':
            for word in self.spam_prob:
                if word in text:
                    predict_spam += np.log(self.spam_prob[word])
                else:
                    predict_spam += np.log(1 - self.spam_prob[word])
            for word in self.ham_prob:
                if word in text:
                    predict_ham += np.log(self.ham_prob[word])
                else:
                    predict_ham += np.log(1 - self.ham_prob[word])
        # Multinomial NB
        elif self.classifier_type[0] == 'M':
            for word in text:
                xi = text[word]
                # Multinomial BF
                if self.classifier_type[-2] == 'B':
                    xi = 1
                if word in self.spam_prob:
                    predict_spam += xi * np.log(self.spam_prob[word])
                if word in self.ham_prob:
                    predict_ham += xi * np.log(self.ham_prob[word])
        # return 1 if we classify this email as spam and 0 if not
        if predict_spam > predict_ham:
            return 1
        return 0


In [159]:
#svm

import numpy as np
from sklearn import svm


# svm classifier
class SVM:
    def __init__(self, kernel):
        self.clf = svm.SVC(kernel=kernel)
        self.feature_indices = {}

    def fit(self, texts, features):
        # transform dictionary to vector
        self.feature_indices = {features[i]: i for i in range(len(features))}
        train_data = np.zeros((len(texts), len(self.feature_indices)))
        labels = np.zeros((len(texts),))
        for i, (word_count, is_spam) in enumerate(texts):
            for word, count in word_count.items():
                if word in self.feature_indices:
                    train_data[i, self.feature_indices[word]] = count
            labels[i] = is_spam
        self.clf.fit(train_data, labels)

    def predict(self, text):
        test_data = np.zeros((1, len(self.feature_indices)))
        for word, count in text.items():
            if word in self.feature_indices:
                test_data[0, self.feature_indices[word]] = count
        label = self.clf.predict(test_data)
        return int(np.squeeze(label))




In [164]:
#adversial

import numpy as np
import os


# the adversary that tries to cheat the NB classifier model by adding words
class Attacker:
    def __init__(self, model):
        if model.classifier_type != 'Multinomial BF':
            raise ValueError('Invalid classifier type.')
        self.features = []
        self.log_odds = []
        for word in model.spam_prob:
            self.features.append(word)
            self.log_odds.append(int((np.log(model.spam_prob[word]) - np.log(model.ham_prob[word])) * 100))

    def attack(self, text, max_cost):
        one_hot = self.text2vec(text)
        cost, add_words = self.get_list(one_hot, max_cost)
        if add_words is not None:
            for i in add_words:
                text[self.features[i - 1]] = 1
            return cost
        return 0

    def get_list(self, one_hot, max_cost):
        gap = self.get_gap(one_hot)
        add_list = None
        cost, add_indices = self.find_mcc(one_hot, len(one_hot), gap)
        if gap > 0 and cost <= max_cost:
            add_list = add_indices
        return cost, add_list

    def text2vec(self, text):
        one_hot = []
        for word in self.features:
            if word in text:
                one_hot.append(1)
            else:
                one_hot.append(0)
        return one_hot

    def get_gap(self, one_hot):
        gap = 0
        for i, present in enumerate(one_hot):
            if present == 1:
                gap += self.log_odds[i]
        return gap

    # find the minimum cost to change the text to non-spam
    def find_mcc(self, one_hot, i, gap):
        if gap <= 0:
            return 0, []
        if i == 0:
            return np.inf, None
        min_cost = np.inf
        min_list = None
        if one_hot[i - 1] == 0 and self.log_odds[i - 1] < 0:
            cur_cost, cur_list = self.find_mcc(one_hot, i - 1, gap + self.log_odds[i - 1])
            if cur_cost + 1 < min_cost:
                min_cost = cur_cost + 1
                min_list = cur_list
                min_list.append(i)
        else:
            cur_cost, cur_list = self.find_mcc(one_hot, i - 1, gap)
            if cur_cost < min_cost:
                min_cost = cur_cost
                min_list = cur_list
        return min_cost, min_list


class Defender(NB):
    def __init__(self):
        super(Defender, self).__init__('Multinomial BF')
        self.adversary = Attacker(self)

    def fit(self, texts, features):
        super(Defender, self).fit(texts, features)
        self.adversary = Attacker(self)

    def predict(self, text):
        one_hot = self.adversary.text2vec(text)
        originals = self.get_original(one_hot)
        ###
        spam_prob_vec = list(self.spam_prob.values())
        ham_prob_vec = list(self.ham_prob.values())
        print(spam_prob_vec)
        print(ham_prob_vec)
        predict_spam = 0
        predict_ham = 0
        exp_predict_spam = 0
        for i in range(len(one_hot)):
            if one_hot[i] == 1:
                predict_spam += np.log(spam_prob_vec[i])
        for i in range(len(one_hot)):
            if one_hot[i] == 1:
                predict_ham += np.log(ham_prob_vec[i])
        if predict_spam <= predict_ham or len(originals) == 0:
            exp_predict_spam += np.exp(predict_spam)
        for original in originals:
            predict_original = 0
            for i in range(len(original)):
                if original[i] == 1:
                    predict_original += np.log(spam_prob_vec[i])
            exp_predict_spam += np.exp(predict_original)
        if exp_predict_spam > np.exp(predict_ham):
            return 1
        return 0

    def get_original(self, one_hot):
        originals = []
        for i in range(len(one_hot)):
            if one_hot[i] == 1:
                one_hot[i] = 0
                _, mcc_list = self.adversary.get_list(one_hot, 1)
                if mcc_list is not None and mcc_list[0] == i + 1:
                    originals.append(one_hot.copy())
                one_hot[i] = 1
        return originals


In [165]:
import os
import pandas as pd

if __name__ == '__main__':
    # load data
    base_path = 'lingspam_public/lemm_stop'
    print(base_path)
    train_emails = []
    for i in range(1, 10):
        train_emails.extend(load_emails(os.path.join(base_path, 'part' + str(i))))
    test_emails = load_emails(os.path.join(base_path, 'part10'))
    # top 10 features
    top_10 = select_feature(train_emails, 10)
    print('top 10 features:', top_10)

    # create naive bayes models
    nb_models = {
        'Binomial NB with BF': NB('Binomial BF'),
        'Multinomial NB with BF': NB('Multinomial BF'),
        'Multinomial NB with TF': NB('Multinomial TF')
    }
    # train ans test models
    nb_results = {}
    for feature_size in [10, 100, 1000]:
        for model_name, model in nb_models.items():
            # select features
            features = select_feature(train_emails, feature_size)
            model.fit(train_emails, features)
            precision, recall, _, _ = evaluate(model, test_emails)
            nb_results[model_name + ', N=' + str(feature_size)] = [precision, recall]
    nb_results_df = pd.DataFrame(nb_results, index=['precision', 'recall']).T
    # print results
    print(nb_results_df)

    # create svm models
    svm_models = {
        'Linear SVM': SVM('linear'),
        'RBF kernel SVM': SVM('rbf')
    }
    # train ans test models
    svm_results = {}
    for feature_size in [10, 100, 1000]:
        for model_name, model in svm_models.items():
            # select features
            features = select_feature(train_emails, feature_size)
            model.fit(train_emails, features)
            precision, recall, _, _ = evaluate(model, test_emails)
            svm_results[model_name + ', N = ' + str(feature_size)] = [precision, recall]
    svm_results_df = pd.DataFrame(svm_results, index=['Precision', 'Recall']).T
    # print results
    print(svm_results_df)

    # adversarial test
    baseline = NB('Multinomial BF')
    baseline.fit(train_emails, top_10)
    _, _, _, base_bnr_before = evaluate(baseline, test_emails)
    print('Before attack:')
    print('False negative rate of the baseline NB classifier = ', base_bnr_before)
    # attack launched
    attacker = Attacker(baseline)
    total_cost = 0
    for email, _ in test_emails:
        total_cost += attacker.attack(email, 10)
    print('Attack Launched......')
    print('Average cost by attacker = ', total_cost / len(test_emails))
    _, _, _, base_fnr_after = evaluate(baseline, test_emails)
    print('After attack:')
    print('False negative rate of the baseline NB classifier = ', base_fnr_after)
    # update the classifier to defend the attack
    updated = Defender()
    updated.fit(train_emails, top_10)
    _, _, updated_fpr, updated_fnr = evaluate(updated, test_emails)
    print('Classifier updated......')
    print('False negative rate of the updated NB classifier = ', updated_fnr)
    print('False positive rate of the updated NB classifier = ', updated_fpr)


lingspam_public/lemm_stop
top 10 features: ['language', 'remove', 'free', 'linguistic', 'university', 'money', 'click', 'market', 'our', 'business']
                                precision    recall
Binomial NB with BF, N=10        0.803571  0.918367
Multinomial NB with BF, N=10     0.803571  0.918367
Multinomial NB with TF, N=10     0.836364  0.938776
Binomial NB with BF, N=100       1.000000  0.755102
Multinomial NB with BF, N=100    0.960000  0.979592
Multinomial NB with TF, N=100    0.940000  0.959184
Binomial NB with BF, N=1000      1.000000  0.612245
Multinomial NB with BF, N=1000   1.000000  0.938776
Multinomial NB with TF, N=1000   1.000000  0.938776


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this 

                          Precision    Recall
Linear SVM, N = 10         0.906977  0.795918
RBF kernel SVM, N = 10     0.900000  0.734694
Linear SVM, N = 100        0.952381  0.816327
RBF kernel SVM, N = 100    0.972222  0.714286
Linear SVM, N = 1000       0.807692  0.857143
RBF kernel SVM, N = 1000   1.000000  0.591837
Before attack:
False negative rate of the baseline NB classifier =  0.08163265306122448
Attack Launched......
Average cost by attacker =  0.3230240549828179
After attack:
False negative rate of the baseline NB classifier =  0.9591836734693877


TypeError: 'list' object is not callable